<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/Stcok_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bta-lib
!pip install ta
!pip install yfinance

In [ ]:
import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator

start="2022-01-01"
end="2023-06-20"
stock = "THYAO.IS"
data = yf.download(stock, start=start, end=end)
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-07,172.300003,174.800003,170.500000,172.500000,172.500000,58786669
2023-06-08,171.600006,173.000000,168.600006,171.600006,171.600006,43644443
2023-06-09,172.300003,178.100006,171.100006,176.899994,176.899994,71672679
2023-06-12,178.300003,185.800003,177.600006,183.300003,183.300003,64378620
2023-06-14,177.699997,182.899994,177.399994,181.000000,181.000000,40334729


In [ ]:
# Below 30 is oversold, upper 70 is overbought
data["RSI"] = btalib.rsi(data["Close"], period=14).df

#If little period conflicts with the bigger one it means buy else sell.
data["SMA9"] = btalib.sma(data['Close'], period=9).df
data["SMA20"] = btalib.sma(data['Close'], period=20).df

#A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

#When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
#Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
data["MACD"] = btalib.macd(data['Close']).df["macd"]
data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

#Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
#This suggests that the stock may be oversold, and a potential price rebound is expected.
#Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
#This indicates that the stock may be overbought, and a potential price correction is expected.
data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

#When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
#from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
data["STDEV"] = data["Close"].rolling(window=10).std()

#Generate a buy signal when the price crosses above the PSAR.
#This indicates a potential reversal in the downward trend and suggests a buying opportunity.
data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

#Williams
#Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
#The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
#Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
#This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

#The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.10/dist-packages/btalib/meta/lines.py:302: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trailer = trailprefix.append(series[p2:])
/usr/local/lib/python3.1

In [ ]:
data = data.dropna()

In [ ]:
scores = []
for row in range(data.shape[0]):
  score = 0

  if data["RSI"][row] < 30:
    score += 1
  else:
    score+=(100-int(data["RSI"][row]))/70

  if data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
    score += 1
  else:
    score += -1

  if data["STOCH-D"][row] <= 20:
    score += 1
  elif data["STOCH-D"][row] >= 80:
    score += -1

  #TODO: Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  if data["MACD"][row] > data["SIGNAL"][row]:
    score+=1
  elif data["MACD"][row] < data["SIGNAL"][row]:
    score+=-1

  if data["BB-LOWER"][row] > data["Close"][row]:
    score += 1
  elif data["BB-MID"][row] < data["Close"][row]:
    score += +0.25
  elif data["BB-MID"][row] > data["Close"][row]:
    score += -0.25
  else:
    score += -1

  if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
    score+=1
  else:
    score+=-1

  if data["PSAR"][row] < data["Close"][row]:
    score+=1
  else:
    score+=-1

  #-20 as the overbought threshold and -80 as the oversold threshold.
  if data["WILLIAMS"][row] > -20:
    score+=-1
  elif data["WILLIAMS"][row] < -80:
    score+=1

  if data["AROON"][row] > 0:
    score+=-1
  else:
    score+=1

  scores.append(score)

data["SCORE"] = scores

<ipython-input-259-4377df0d2635>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SCORE"] = scores


In [ ]:
data = data.dropna()

In [ ]:
scores = []

for row in range(data.shape[0]):
  score = 0

  if data["RSI"][row] < 30:
    score += 1
  else:
    score+=(100-int(data["RSI"][row]))/70

  if data["SMA9"][row] > data["SMA20"][row]:
    score+=1
  elif data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
    score+=1
  else:
    score+=-1

  if data["STOCH-D"][row] <= 20:
    score += 1
  elif data["STOCH-D"][row] >= 80:
    score += -1

  if data["MACD"][row] > data["SIGNAL"][row]:
    score+=1
  if data["MACD"][row] > data["SIGNAL"][row] and data["HISTOGRAM"][row-1] < 0 and data["HISTOGRAM"][row] > 0:
    score+=1
  elif data["MACD"][row] < data["SIGNAL"][row]:
    score+=-1

  if data["BB-LOWER"][row] > data["Close"][row]:
    score += 1
  elif data["BB-MID"][row] < data["Close"][row]:
    score += +0.25
  elif data["BB-MID"][row] > data["Close"][row]:
    score += -0.25
  else:
    score += -1

  if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
    score+=1
  else:
    score+=-1

  if data["PSAR"][row] < data["Close"][row]:
    score+=1
  else:
    score+=-1

  #-20 as the overbought threshold and -80 as the oversold threshold.
  if data["WILLIAMS"][row] > -20:
    score+=-1
  elif data["WILLIAMS"][row] < -80:
    score+=1

  if data["AROON"][row] > 0:
    score+=-1
  else:
    score+=1

  scores.append(score)

data["SCORE"] = scores

<ipython-input-260-f67379de50e6>:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
output_file = stock + ":" + start + ":" + end + '.xlsx'
writer = pd.ExcelWriter(output_file)
data.to_excel(writer, index=False, sheet_name=stock)
writer.save()

45.82195938024353


0.0

In [ ]:
tr_balance = 2000
stock_balance = 0

transactions = []

for row in range(data.shape[0]):
  if data["SCORE"][row] >= 4.5 and tr_balance > 100:
   stock_balance = tr_balance / data["Close"][row]
   tr_balance-= stock_balance*data["Close"][row]
   transactions.append(["BUY", data.index[row], data["Close"][row], stock_balance, tr_balance])

  elif data["SCORE"][row] <= -3 and stock_balance > 0:
    tr_balance+=stock_balance*data["Close"][row]
    stock_balance-=stock_balance
    transactions.append(["SELL", data.index[row], data["Close"][row], stock_balance, tr_balance])


print(stock_balance)
tr_balance

In [ ]:
transactions_df = pd.DataFrame(transactions, columns=["B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE"])
transactions_df